<a href="https://colab.research.google.com/github/Abdallah-M-Ali/Machine-Learning-with-Python-Projects/blob/main/Book_recommendation_knn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

In [ ]:
!pip install fuzzywuzzy

In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
# from fuzzywuzzy import process

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2022-05-12 10:02:45--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  38.2MB/s    in 0.7s    

2022-05-12 10:02:46 (38.2 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
print(df_books.shape)
df_books

In [ ]:
df_books['isbn'][103196]
df_books.loc[103196]

In [ ]:
print(df_ratings.shape)
df_ratings

In [ ]:
df_ratings['isbn'].value_counts()

**The first method of cleaning the data, which didn't work well**

In [ ]:
isbn_list = df_ratings['isbn'].value_counts().loc[lambda x : x >100]
isbn_list = isbn_list.index.tolist()
print(len(isbn_list))
isbn_list

In [ ]:
df_ratings.loc[df_ratings['isbn'] == '0971880107']

In [ ]:
book_higher_rate = df_ratings[df_ratings['isbn'].isin(isbn_list)]
book_higher_rate.loc[book_higher_rate['isbn'] == '05162443314']
print(len(book_higher_rate))
book_higher_rate

In [ ]:
clean_data = book_higher_rate[book_higher_rate['user'].isin(user_list)]
print(len(clean_data['isbn'].value_counts()))
clean_data

In [ ]:
clean_data.info()

In [ ]:
rating_data = clean_data.pivot(index='isbn', columns='user', values='rating').fillna(0)
ind = rating_data.index.tolist()
print(ind.index('002542730X'))
rating_data

In [ ]:
final_book_list = clean_data['isbn'].unique().tolist()

In [ ]:
books = df_books[df_books['isbn'].isin(final_book_list)]
books.loc[books['title'] == "Where the Heart Is (Oprah's Book Club (Paperback))"]['isbn'].item()

In [ ]:
mat_csr = csr_matrix(rating_data.values)
mat_csr

***The second method, online checked***

In [ ]:
df = pd.merge(df_ratings, df_books, on='isbn')
print(len(df_ratings['isbn'].unique()), ' ids in ratings data')
print(len(df_books['isbn'].unique()), ' ids on bookd data', len(df_books['title'].unique()), ' books in bookd data')
print(len(df['isbn'].unique()), ' ids in mereged data', len(df['title'].unique()), ' books in merged data')
df['user'].isnull().sum()
douple_id = df_books[['title', 'isbn']].value_counts().reset_index()
douple_id = douple_id.drop_duplicates(subset=['title'])
len(douple_id['isbn'].unique())
clean_id = douple_id['isbn'].tolist()

340556  ids in ratings data
271379  ids on bookd data 242154  books in bookd data
270170  ids in mereged data 241090  books in merged data


In [ ]:
exp1 = df_books[df_books['isbn'].isin(clean_id)]
exp1
# exp1.rename(coulumns= {0: 'count'})

,isbn,title,author
0,0195153448,Classical Mythology,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber
...,...,...,...
271374,0440400988,There's a Bat in Bunk Five,Paula Danziger
271375,0525447644,From One to One Hundred,Teri Sloat
271376,006008667X,Lily Dale : The True Story of the Town that Ta...,Christine Wicker
271377,0192126040,Republic (World's Classics),Plato


In [ ]:
df3 = pd.merge(df_ratings, exp1, on='isbn')
print(len(exp1['isbn'].unique()), ' ids on bookd data', len(exp1['title'].unique()), ' books in bookd data')
print(len(df3['isbn'].unique()), ' ids in mereged data', len(df3['title'].unique()), ' books in merged data')
df3.loc[df3['title'] == 'The Surgeon']

242154  ids on bookd data 242154  books in bookd data
241041  ids in mereged data 241041  books in merged data


,user,isbn,rating,title,author
767361,239551,0749902698,4.0,The Surgeon,Francis Roe


In [ ]:
book_ratingCount = (df3.groupby(by = ['title'])['rating'].
                    count().reset_index().
                    rename(columns = {'rating': 'booklRaingsCount'}))
book_ratingCount

,title,booklRaingsCount
0,A Light in the Storm: The Civil War Diary of ...,4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, ...",1
4,Beyond IBM: Leadership Marketing and Finance ...,1
...,...,...
241036,Ã?Â?lpiraten.,2
241037,Ã?Â?rger mit Produkt X. Roman.,4
241038,Ã?Â?sterlich leben.,1
241039,Ã?Â?stlich der Berge.,3


In [ ]:
rating_with_totalCount = df3.merge(book_ratingCount, left_on = 'title', right_on= 'title', how = 'left')
rating_with_totalCount

,user,isbn,rating,title,author,booklRaingsCount
0,276725,034545104X,0.0,Flesh Tones: A Novel,M. J. Rose,60
1,2313,034545104X,5.0,Flesh Tones: A Novel,M. J. Rose,60
2,6543,034545104X,0.0,Flesh Tones: A Novel,M. J. Rose,60
3,8680,034545104X,5.0,Flesh Tones: A Novel,M. J. Rose,60
4,10314,034545104X,9.0,Flesh Tones: A Novel,M. J. Rose,60
...,...,...,...,...,...,...
786829,276688,0425150526,0.0,Death Crosses the Border,Janice Steinberg,1
786830,276688,0449907422,0.0,Jazz Funeral: A Skip Langdon Novel,Julie Smith,1
786831,276690,0590907301,0.0,Triplet Trouble and the Class Trip (Triplet Tr...,Debbie Dadey,1
786832,276704,0679752714,0.0,A Desert of Pure Feeling (Vintage Contemporaries),Judith Freeman,1


In [ ]:
user_ratingCount = (df3.groupby(by = ['user'])['rating'].
                    count().reset_index().
                    rename(columns = {'rating': 'userRaingsCount'}))
user_ratingCount

,user,userRaingsCount
0,2,1
1,8,14
2,9,2
3,10,1
4,12,1
...,...,...
77395,278843,48
77396,278844,2
77397,278849,4
77398,278851,21


In [ ]:
df_merged = rating_with_totalCount.merge(user_ratingCount, left_on = 'user', right_on= 'user', how = 'left')
print(df_merged.shape)
df_final = df_merged.loc[(df_merged['booklRaingsCount'] > 100) & (df_merged['userRaingsCount'] > 200)]
print(len(df_merged) - len(df_final))
df_final

(786834, 7)
765916


,user,isbn,rating,title,author,booklRaingsCount,userRaingsCount
63,278418,0446520802,0.0,The Notebook,Nicholas Sparks,116,3547
65,3363,0446520802,0.0,The Notebook,Nicholas Sparks,116,714
66,7158,0446520802,10.0,The Notebook,Nicholas Sparks,116,258
69,11676,0446520802,10.0,The Notebook,Nicholas Sparks,116,8747
74,23768,0446520802,6.0,The Notebook,Nicholas Sparks,116,1293
...,...,...,...,...,...,...,...
316293,236283,0515135739,0.0,Eleventh Hour: An FBI Thriller (FBI Thriller (...,Catherine Coulter,102,887
316298,251613,0515135739,0.0,Eleventh Hour: An FBI Thriller (FBI Thriller (...,Catherine Coulter,102,874
316299,252071,0515135739,0.0,Eleventh Hour: An FBI Thriller (FBI Thriller (...,Catherine Coulter,102,272
316302,256407,0515135739,0.0,Eleventh Hour: An FBI Thriller (FBI Thriller (...,Catherine Coulter,102,437


In [ ]:
result_list = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
df_merged[df_merged['title'].isin(result_list)]
df_merged.loc[df_merged['title'] == "I'll Be Seeing You"]
df_final.loc[df_final['title'] == "I'll Be Seeing You"]

,user,isbn,rating,title,author,booklRaingsCount,userRaingsCount
45134,11601,0671888587,9.0,I'll Be Seeing You,Mary Higgins Clark,140,1226
45135,11676,0671888587,8.0,I'll Be Seeing You,Mary Higgins Clark,140,8747
45136,12982,0671888587,8.0,I'll Be Seeing You,Mary Higgins Clark,140,216
45137,13273,0671888587,8.0,I'll Be Seeing You,Mary Higgins Clark,140,334
45138,16795,0671888587,0.0,I'll Be Seeing You,Mary Higgins Clark,140,2200
...,...,...,...,...,...,...,...
45261,265115,0671888587,0.0,I'll Be Seeing You,Mary Higgins Clark,140,878
45263,271195,0671888587,0.0,I'll Be Seeing You,Mary Higgins Clark,140,210
45264,271284,0671888587,0.0,I'll Be Seeing You,Mary Higgins Clark,140,867
45265,273979,0671888587,0.0,I'll Be Seeing You,Mary Higgins Clark,140,217


In [ ]:
print(len(df_final['title'].unique()), ' books')
print(len(df_final['user'].unique()), ' users')
print(len(df_final['isbn'].unique()), ' book_ids')

396  books
572  users
396  book_ids


In [ ]:
book_threshold = 100
user_threshold = 200
df_final2 = df_merged.query('totalRaingsCount > @book_threshold')
df_final2 = df_final2.query('userRaingsCount > @user_threshold')
df_final2

In [ ]:
finalDf_pivot = df_final.pivot_table(index='title', columns='user', values='rating').fillna(0)
finalDf_pivot

user,254,2276,2766,3363,4385,6251,6543,6575,7158,7286,...,271284,273979,274004,274061,274301,274308,275970,277427,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4 Blondes,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A Case of Need,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wild Animus,0.0,0.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Winter Moon,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Year of Wonders,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
book_names = finalDf_pivot.index.tolist()
book_names

['1st to Die: A Novel',
 '2nd Chance',
 '4 Blondes',
 'A Beautiful Mind: The Life of Mathematical Genius and Nobel Laureate John Nash',
 'A Case of Need',
 'A Confederacy of Dunces (Evergreen Book)',
 'A Girl Named Zippy: Growing Up Small in Mooreland Indiana (Today Show Book Club #3)',
 'A Heartbreaking Work of Staggering Genius',
 'A Is for Alibi (Kinsey Millhone Mysteries (Paperback))',
 'A Lesson Before Dying (Vintage Contemporaries (Paperback))',
 'A Light in the Window (The Mitford Years)',
 'A Prayer for Owen Meany',
 'A Thin Dark Line (Mysteries &amp; Horror)',
 "A Thousand Acres (Ballantine Reader's Circle)",
 "A Virtuous Woman (Oprah's Book Club (Paperback))",
 'A Walk in the Woods: Rediscovering America on the Appalachian Trail (Official Guides to the Appalachian Trail)',
 'A Widow for One Year',
 'A Wrinkle In Time',
 'About a Boy (Movie Tie-In)',
 'All I Really Need to Know',
 'All That Remains (Kay Scarpetta Mysteries (Paperback))',
 'All the Pretty Horses (The Border Tri

In [ ]:
idx = book_names.index("Where the Heart Is (Oprah's Book Club (Paperback))")
ex = finalDf_pivot.iloc[idx, :].values.reshape(1, -1)
ex

array([[0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
book_names.index("I'll Be Seeing You")

142

In [ ]:
mat_csr2 = csr_matrix(finalDf_pivot.values)
mat_csr2

<396x572 sparse matrix of type '<class 'numpy.float32'>'
	with 5100 stored elements in Compressed Sparse Row format>

In [ ]:
model = NearestNeighbors(metric= 'cosine', algorithm='brute', n_neighbors=20)
model.fit(mat_csr2)

NearestNeighbors(algorithm='brute', metric='cosine', n_neighbors=20)

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book):
  idx = book_names.index(book)
  # distance, indices = model.kneighbors(mat_csr2[idx], 5)
  distance, indices = model.kneighbors(finalDf_pivot.iloc[idx, :].values.reshape(1, -1), 5)
  book_title = []
  for i in indices[0]:
    bookid = book_names[i]
    book_title.append(bookid)
  pred_result = []
  for (i,j) in zip(book_title[1:], distance[0][1:]):
    value = [i,j]
    pred_result.append(value)
  # pred_result = [book_title[1:], distance[0][1:]]
  recommended_books = [book_title[0], pred_result]

  return recommended_books

In [ ]:
get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")

["Where the Heart Is (Oprah's Book Club (Paperback))",
 [["The Dive From Clausen's Pier (Alex Awards)", 0.7012136],
  ['Hide &amp; Seek', 0.74340963],
  ['Bel Canto: A Novel', 0.74485695],
  ['I Know This Much Is True', 0.748755]]]

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2): 
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["The Dive From Clausen's Pier (Alex Awards)", 0.7012136], ['Hide &amp; Seek', 0.74340963], ['Bel Canto: A Novel', 0.74485695], ['I Know This Much Is True', 0.748755]]]
You haven't passed yet. Keep trying!


**The test structure has some problem since one of the book is out of the cleaned data**

'The Surgeon' book has less than 100 rating, so it is exluded from training data. this directly shows that data structure of the test is different than the requirments which led to various result.
please kindly see the code below which clarify the book_ratings_count and user_ratings_count of the 'The Surgeon' book

In [ ]:
# original data with count_value of the users and books
# 'The Surgeon' has only 1 bookRatingsCount and 19 userRatingsCount
df_merged.loc[df_merged['title'] == 'The Surgeon']

,user,isbn,rating,title,author,booklRaingsCount,userRaingsCount
767361,239551,0749902698,4.0,The Surgeon,Francis Roe,1,19


In [ ]:
# cleaned data after excluding dataset users with less than 200 ratings and books with less than 100 ratings.
# the book does not belong to this dataframe.
df_final.loc[df_final['title'] == 'The Surgeon']

,user,isbn,rating,title,author,booklRaingsCount,userRaingsCount
